## Duckbot Automated Growth assays code library

In [1]:
import random
import pandas as pd
import DuckbotExptSetupUtils as exp
import os
import json

### Create dataframe with experiment metadata

In [2]:
# USER ACTION REQUIRED

#1. DEFINE EXPERIMENTAL VARIABLES
genotypes = ["Sp7498", "sp3484"] # Replace with names for unique duckweed genotypes
media = ["Mock", "Salt"] # Replace with names for unique media
reps  = 3 # Replace with your desired number of replicates for each duckweed/media combination. 

#2. DEFINE FILE LOCATION AND NAME
expt_setup_parent_dir = os.getcwd() # Default uses current working directory but you can replace with your own choice. 
expt_setup_dir = "TestExptDir" # Name of the folder to hold experiment data and metadata including the setup file
expt_setup_file_name = "TestFile.json" #Name for the experiment setup file (Metadata)

        

In [3]:
# Creates master list of sample info, shuffles and then assigns to plates and wells. 
master_expt_list = []

for g in genotypes:
    for m in media:
        for x in range(reps):
             master_expt_list.append({"genotype": g, "media": m, "condition_replicate": x + 1})


random.shuffle(master_expt_list)
master_expt_list = exp.assign_plates_and_wells(master_expt_list)
expt_dict = {"sample_info" : master_expt_list}




In [4]:
print(expt_dict["sample_info"])

[{'genotype': 'Sp7498', 'media': 'Salt', 'condition_replicate': 2, 'plate_well_id': 'Plate_1_Well_A1', 'Plate': 'Plate_1', 'Well': 'A1'}, {'genotype': 'Sp7498', 'media': 'Mock', 'condition_replicate': 2, 'plate_well_id': 'Plate_1_Well_A2', 'Plate': 'Plate_1', 'Well': 'A2'}, {'genotype': 'sp3484', 'media': 'Salt', 'condition_replicate': 3, 'plate_well_id': 'Plate_1_Well_A3', 'Plate': 'Plate_1', 'Well': 'A3'}, {'genotype': 'Sp7498', 'media': 'Salt', 'condition_replicate': 1, 'plate_well_id': 'Plate_1_Well_A4', 'Plate': 'Plate_1', 'Well': 'A4'}, {'genotype': 'Sp7498', 'media': 'Mock', 'condition_replicate': 1, 'plate_well_id': 'Plate_1_Well_A5', 'Plate': 'Plate_1', 'Well': 'A5'}, {'genotype': 'sp3484', 'media': 'Mock', 'condition_replicate': 1, 'plate_well_id': 'Plate_1_Well_A6', 'Plate': 'Plate_1', 'Well': 'A6'}, {'genotype': 'Sp7498', 'media': 'Mock', 'condition_replicate': 3, 'plate_well_id': 'Plate_1_Well_B1', 'Plate': 'Plate_1', 'Well': 'B1'}, {'genotype': 'sp3484', 'media': 'Mock', 

In [4]:
#Create folder and save experimental set up file to it

path = os.path.join(expt_setup_parent_dir, expt_setup_dir)

os.mkdir(path)
os.chdir(path)

with open(expt_setup_file_name, 'w') as f:
    json.dump(expt_dict, f)

In [6]:
# Turn samples list into a dataframe

#Import from file (in case user wants to make any manual edits to the JSON file after creating it)

#TO DO import from Json
with open(expt_setup_file_name) as datafile:
    expt_data = json.load(datafile)

sample_data = expt_data["sample_info"]
df = pd.DataFrame(sample_data)
print (df[0:])


   genotype media  condition_replicate    plate_well_id    Plate Well
0    sp3484  Mock                    3  Plate_1_Well_A1  Plate_1   A1
1    sp3484  Salt                    2  Plate_1_Well_A2  Plate_1   A2
2    Sp7498  Mock                    1  Plate_1_Well_A3  Plate_1   A3
3    Sp7498  Salt                    1  Plate_1_Well_A4  Plate_1   A4
4    Sp7498  Mock                    3  Plate_1_Well_A5  Plate_1   A5
5    sp3484  Mock                    1  Plate_1_Well_A6  Plate_1   A6
6    sp3484  Mock                    2  Plate_1_Well_B1  Plate_1   B1
7    sp3484  Salt                    3  Plate_1_Well_B2  Plate_1   B2
8    sp3484  Salt                    1  Plate_1_Well_B3  Plate_1   B3
9    Sp7498  Salt                    3  Plate_1_Well_B4  Plate_1   B4
10   Sp7498  Mock                    2  Plate_1_Well_B5  Plate_1   B5
11   Sp7498  Salt                    2  Plate_1_Well_B6  Plate_1   B6


### Set up physical plates

#### Label Plates and add plates to machine

In [10]:
num_plates = df.Plate.nunique()

print("This experiment requires {} 24-well plate(s)".format(num_plates))
print("----")
lst = list(range(1,num_plates + 1))
for n in lst:
    print("Label a plate with experiment ID or initials and 'plate {}'".format(n))
print("----")
print ("Place the 24-well plate(s) in the jubilee".format(num_plates))
print ("Start at position 1 and fill empty plate slots in order")



This experiment requires 1 24-well plate(s)
----
[1]
Label a plate with experiment ID or initials and 'plate 1'
----
Place the 24-well plate(s) in the jubilee
Start at position 1 and fill empty plate slots in order


#### Add media to plates
When prompted insert containers of the relevant sterile media into the input slot on the Jubilee. 

In [5]:
media_df = df.groupby('media')
for media_type, sample_df in media_df:
    print("Place container of media type **{0}** into jubilee and ensure lid is open".format(media_type))
    print("""Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing""")
    input() 
    for index,s in sample_df.iterrows():
        #move to plate and well
        print("Move to {0}, well {1}".format(s["Plate"], s["Well"]))
        print("Dispensing media of type {0} into {1}, well {2}".format(media_type,s["Plate"], s["Well"]))
    print("---")


Place container of media type **Mock** into jubilee and ensure lid is open
Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing
ok
Move to Plate_1, well A3
Dispensing media of type Mock into Plate_1, well A3
Move to Plate_1, well B2
Dispensing media of type Mock into Plate_1, well B2
Move to Plate_1, well B3
Dispensing media of type Mock into Plate_1, well B3
Move to Plate_1, well B4
Dispensing media of type Mock into Plate_1, well B4
Move to Plate_1, well B5
Dispensing media of type Mock into Plate_1, well B5
Move to Plate_1, well B6
Dispensing media of type Mock into Plate_1, well B6
Place container of media type **Salt** into jubilee and ensure lid is open
Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing
ok
Move to Plate_1, well A1
Dispensing media of type Salt into Plate_1, well A1
Move to Plate_1, well A2
Dispensing media of 

#### Add duckweed to wells
Place a container filled with fronds of the relevant duckweed type and the machine will attempt to move individual fronds into the relevant wells. After each attempt at filling all wells a camera will take pictures of each well to confirm success and then unsuccessful wells will be reattempted

In [10]:
grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    print("""Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing""")
    input() 
    for index,s in sample_df.iterrows():
        #move to plate and well
        print("Move to {0}, well {1}".format(s["Plate"], s["Well"]))
        print("Dispensing media of type {0} into {1}, well {2}".format(field_value,s["Plate"], s["Well"]))


Place container of duckweed type **Sp7498** into jubilee and ensure lid is open
Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing
ok
Move to Plate_1, well A2
Dispensing media of type Sp7498 into Plate_1, well A2
Move to Plate_1, well A3
Dispensing media of type Sp7498 into Plate_1, well A3
Move to Plate_1, well A4
Dispensing media of type Sp7498 into Plate_1, well A4
Move to Plate_1, well B1
Dispensing media of type Sp7498 into Plate_1, well B1
Move to Plate_1, well B2
Dispensing media of type Sp7498 into Plate_1, well B2
Move to Plate_1, well B5
Dispensing media of type Sp7498 into Plate_1, well B5
Place container of duckweed type **sp3484** into jubilee and ensure lid is open
Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing
ok
Move to Plate_1, well A1
Dispensing media of type sp3484 into Plate_1, well A1
Move to Plate_1, well